In [92]:
import json
import xlsxwriter
import pandas as pd
import os

In [124]:
lang_pair = "fin_heb"
filetype = "gold" # parallel_sample | gold
target_folder = f"alternative_data/{lang_pair}/source_excel_{filetype}"

In [118]:
os.makedirs(target_folder, exist_ok=True)

In [119]:
with open(f"alternative_data/{lang_pair}/{filetype}_augmented.json", "r") as f:
    sentences = json.load(f)

In [120]:
real_sentences = []
temp_ids = set()
for sent in sentences:
    if sent["sentence_id"] not in temp_ids:
        temp_ids.add(sent["sentence_id"])
        real_sentences.append({"sentence":sent["masked_sentence"].replace("[MASK]", sent["word"]),
                              "sentence_id": sent["sentence_id"],
                              "word_id": -1,
                              "probability": 0})

for sent in sentences:
    ct = 0
    for rep in sent["replacement"]:
        if rep["word"].strip().lower()!=sent["word"].lower():
            ct+=1
            real_sentences.append({"sentence":sent["masked_sentence"].replace("[MASK]", rep["word"].strip()),
                                  "sentence_id": sent["sentence_id"],
                                  "word_id": sent["word_id"],
                                  "probability": rep["prob"]})

df = pd.DataFrame(real_sentences).sort_values(["sentence_id", "word_id"]).reset_index(drop=True)
df

,sentence,sentence_id,word_id,probability
0,"Salomon , Daavidin pojan , Israelin kuninkaan ...",0,-1,0.000
1,"Daavidin , Daavidin pojan , Israelin kuninkaan...",0,0,0.621
2,"Jeesuksen , Daavidin pojan , Israelin kuninkaa...",0,0,0.171
3,"Jumalan , Daavidin pojan , Israelin kuninkaan ...",0,0,0.063
4,"Mooseksen , Daavidin pojan , Israelin kuninkaa...",0,0,0.061
...,...,...,...,...
2128756,Hän sai panna pois vangin puvun ja aina aterio...,22290,15,0.031
2128757,Hän sai panna pois vangin puvun ja aina aterio...,22290,16,0.005
2128758,Hän sai panna pois vangin puvun ja aina aterio...,22290,16,0.004
2128759,Hän sai panna pois vangin puvun ja aina aterio...,22290,16,0.002


In [121]:
selected_sentences = sorted(real_sentences, key=lambda d: (d['sentence_id'], d["word_id"])) 

In [122]:
for s in selected_sentences:
    if "\n" in s["sentence"]:
        raise Exception

In [123]:
cur_len = 0
current_sentences = []
current_idx = 0
threshold = 10000000
for s in selected_sentences:
    sentence = s["sentence"]
    if cur_len+len(sentence)>threshold:
        print(cur_len)
        workbook = xlsxwriter.Workbook(f"{target_folder}/{current_idx}.xlsx")
        worksheet = workbook.add_worksheet()
        print(len(current_sentences))
        for i, el in enumerate(current_sentences):
            worksheet.write(f'A{i+1}', el)

        workbook.close()

        current_sentences = [sentence]
        current_idx += 1
        cur_len = len(sentence)
    else:
        current_sentences.append(sentence)
        cur_len += len(sentence)
workbook = xlsxwriter.Workbook(f"{target_folder}/{current_idx}.xlsx")
worksheet = workbook.add_worksheet()
print(len(current_sentences))
for i, el in enumerate(current_sentences):
    worksheet.write(f'A{i+1}', el)

workbook.close()


9999889
86288
9999947
59671
9999959
98980
9999934
81293
9999912
66209
9999966
66296
9999849
64975
9999834
58608
9999928
57565
9999905
51291
9999951
55392
9999889
57174
9999896
60793
9999973
55846
9999948
63649
9999877
66515
9999937
63873
9999826
52124
9999963
49760
9999869
64641
9999964
64385
9999795
65113
9999830
55751
9999933
57750
9999934
56387
9999782
55759
9999999
64812
9999844
62761
9999923
57632
9999811
55175
9999842
87153
9999948
54237
9999886
52709
9999977
52214
5980
